# 새 데이터 가져오기

In [28]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd

In [29]:
head = {'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0'}
data = []
with requests.Session() as s:
    for page in range(1,3):
        url = f"https://www.saramin.co.kr/zf_user/jobs/list/job-category?cat_mcls=2&page={page}&page_count=1000&sort=RD"
        r = s.get(url, headers=head)
        data.append(r)
        print(page)
        if r.status_code != 200:
            break


1
2


In [35]:
item_list = []
for r in data:
    soup = BeautifulSoup(r.text)
    box_item_list = soup.select('div.box_item')


    for box_item in box_item_list:
        temp = {}
        temp['company_name'] = box_item.select_one('.company_nm > .str_tit').text.strip()
        temp['company_sn'] = box_item.select_one('.interested_corp')['csn']
        temp['is_headhunting'] = '헤드헌팅' in box_item.select_one(".company_nm").text
        temp['job_idx'] = int(box_item.find('button', attrs={'rec_idx': True})['rec_idx'])
        temp['job_title'] = box_item.select_one('.notification_info a.str_tit')['title']
        temp['job_link'] = box_item.select_one('.notification_info a.str_tit')['href']
        temp['job_sectors'] = [i.text for i in box_item.select('.notification_info .job_meta .job_sector span')]
        work_place_element = box_item.select_one('.recruit_info .work_place')
        temp['work_place'] = work_place_element.text if work_place_element is not None else None
        temp['career_info'] = box_item.select_one('.recruit_info .career').text
        temp['education_info'] = box_item.select_one('.recruit_info .education').text
        temp['support_date'] = box_item.select_one('.support_info .date').text
        temp['deadline'] = box_item.select_one('.support_info .deadlines').text
        item_list.append(temp)


# 새 시트 만들기

In [36]:
df_new = pd.DataFrame(item_list)
df_new["update"] = pd.to_datetime("now")
df_new["job_link"] = df_new.job_link.apply(lambda x: f"https://www.saramin.co.kr{x}")


def process_deadline(row):
    x = row["deadline"]

    word = x.split(" ")[0]

    if "일" in word:
        delta = pd.Timedelta(days=int(word.split("일")[0]))
    elif "시간" in word:
        delta = pd.Timedelta(hours=int(word.split("시간")[0]))
    elif "분" in word:
        delta = pd.Timedelta(minutes=int(word.split("분")[0]))

    if "등록" in x:
        row["resistered"] = row["update"] - delta
    elif "수정" in x:
        row["modified"] = row["update"] - delta
    else:
        pass

    return row


df_new = df_new.apply(process_deadline, axis=1)

# eda
## 사명 변경된 적 있는지
```
final_df = df[df.groupby('company_sn')['company_name'].transform('nunique') > 1].sort_values(by='job_idx')[['company_sn', 'company_name']].drop_duplicates()
final_df['name_count'] = final_df.groupby('company_sn').cumcount() + 1
final_df.pivot_table(columns='name_count', index='company_sn', values='company_name', aggfunc='first')
```

## 목록에 표시된 해시태그
- 하지만 각 공고마다 가지고 있는 태그 전체 내용이 아니다.
```
df2 = df[['job_idx','job_sectors']]
df2.explode('job_sectors').groupby('job_sectors').size().sort_values(ascending=False).to_csv('job_sectors.csv')
df.explode('job_sectors').groupby('job_sectors').get_group('크롤링')

## jobsector 데이터가 조회할때마다 달라지는지 확인 필요
- 다섯개 이상 선택해도 리스트에 표시되지 않음
- 지정된 우선순위 순서로 표시되는지 아니면 조회할때마다 무작위로 5개 선택하는지 확인 필요

# 구글 시트에서 이전 시트 가져오기

In [32]:
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

gc = gspread.service_account()
sh=gc.open_by_key('1vkZ7FF1bgACwXKxkmSNemmSNiti4Bd2M177w-8H80ww').sheet1

df_old = get_as_dataframe(sh)
df_old

,career_info,company_name,company_sn,deadline,education_info,is_headhunting,job_idx,job_link,job_sectors,job_title,modified,resistered,support_date,update,work_place
0,5 ~ 15년 · 정규직,(주)커리어케어,VzJPaU1xdHVWcmptUXVCTWJ5cFVoQT09,13분 전 등록,대학교(4년)↑,True,47875769,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"['데이터엔지니어', '머신러닝', 'DW', 'ETL', 'AI(인공지능)']",[국내 디지털 헬스케어기업] 데이터 엔지니어,NaN,2024-03-27 19:45:11,~04.07(일),2024-03-27 19:58:11,서울 강남구
1,3 ~ 5년 · 정규직 외,(주)파인에이치알,dmQ3RVplYUlycmswMEF0dVlIcGtvUT09,22분 전 등록,대학교(4년)↑,True,47875741,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"['기술지원', '웹개발', '웹마스터', '유지보수', 'Git']",[중견그룹] IT본부 웹 시스템 개발/운영/관리,NaN,2024-03-27 19:36:11,~04.15(월),2024-03-27 19:58:11,경기 성남시 분당구
2,3 ~ 10년 · 정규직,(주)파인에이치알,dmQ3RVplYUlycmswMEF0dVlIcGtvUT09,23분 전 등록,대학교(4년)↑,True,47875738,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"['보안컨설팅', '정보보안', '네트워크', '취약점진단', 'ISMS']",[중견 소비재그룹] IT 정보보안 인증 유지 및 운영,NaN,2024-03-27 19:35:11,~04.15(월),2024-03-27 19:58:11,경기 성남시 분당구
3,2 ~ 12년 · 정규직 외,(주)파인에이치알,dmQ3RVplYUlycmswMEF0dVlIcGtvUT09,24분 전 등록,대학교(4년)↑,True,47875728,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"['기술지원', '웹개발', '웹마스터', '유지보수', 'Git']",[중견그룹] 지주사 IT팀 웹서비스 관리,NaN,2024-03-27 19:34:11,~04.15(월),2024-03-27 19:58:11,경기 성남시 분당구
4,3 ~ 8년 · 정규직 외,(주)파인에이치알,dmQ3RVplYUlycmswMEF0dVlIcGtvUT09,25분 전 등록,대학교(4년)↑,True,47875725,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"['SE(시스템엔지니어)', 'ERP', 'SAP', 'ABAP']",[코스닥상장 제조사] IT팀 SAP SCM 운영,NaN,2024-03-27 19:33:11,~04.15(월),2024-03-27 19:58:11,경기 성남시 분당구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,3 ~ 10년 · 정규직,한스컨설팅,Zkc5dWtSYXhYSURDVTd0c0tEMStVQT09,3일 전 등록,대학교(4년)↑,True,47846809,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"['기술지원', 'CISO', 'DBA', '네트워크', '정보통신']","코스닥 상장, 클라우드기업 네트웍엔지니어 대리-과장, 이천근무",NaN,2024-03-24 19:58:11,채용시,2024-03-27 19:58:11,서울 구로구
1996,3 ~ 8년 · 정규직,한스컨설팅,Zkc5dWtSYXhYSURDVTd0c0tEMStVQT09,3일 전 등록,대학교(4년)↑,True,47846799,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"['SAP', 'PO(프로덕트오너)', 'SCM']","코스피 상장, 판교 위치 화장품전문기업 SAP SCM 관리",NaN,2024-03-24 19:58:11,채용시,2024-03-27 19:58:11,경기 성남시 분당구
1997,3 ~ 10년 · 정규직,한스컨설팅,Zkc5dWtSYXhYSURDVTd0c0tEMStVQT09,3일 전 등록,대학교(4년)↑,True,47846794,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"['PL(프로젝트리더)', 'PM(프로젝트매니저)', 'PMO', 'PO(프로덕트오...","코스피 상장, 판교 위치 화장품전문기업 IT 기획 대리-과장급",NaN,2024-03-24 19:58:11,채용시,2024-03-27 19:58:11,경기 성남시 분당구
1998,3 ~ 10년 · 정규직,한스컨설팅,Zkc5dWtSYXhYSURDVTd0c0tEMStVQT09,3일 전 등록,대학교(4년)↑,True,47846790,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"['보안컨설팅', '정보보안', '모의해킹', '방화벽', 'ISMS']","코스피 상장, 판교 위치 화장품전문기업 IT 정보보안 대리-과장",NaN,2024-03-24 19:58:11,채용시,2024-03-27 19:58:11,경기 성남시 분당구


# 새시트, 이전시트 병합
- jobsector 열 리스트 확장
- 제목 변경되는 경우 특이 케이스로 보고 따로 표시
- 수정일 : 새 시트 우선
- 작성일 : 이전 시트 우선, 원래 없다가 새로 생겨나는 경우 없을듯
- update 열 : 마지막으로 조회한 시간, 새 시트 우선
- 어떻게 합치는게 세련된 방법인지 모르겠다

- old에서 안쓰는 열은 드롭? 안돼 그러면 아우터가 아니니까

In [34]:
df_new.dtypes


career_info               object
company_name              object
company_sn                object
deadline                  object
education_info            object
is_headhunting              bool
job_idx                   object
job_link                  object
job_sectors               object
job_title                 object
modified          datetime64[ns]
resistered        datetime64[ns]
support_date              object
update            datetime64[ns]
work_place                object
dtype: object

In [37]:
# Merge the dataframes
merged_df = pd.merge(df_old, df_new, how='outer', on=['job_idx'])

# Apply conditions to overlapping parts
# modified new 우선, resistered old 우선, update new 우선

merged_df["resistered"] = merged_df["resistered_x"].fillna(merged_df["resistered_y"])
merged_df["modified"] = merged_df["modified_x"].fillna(merged_df["modified_y"])
merged_df["update"] = merged_df["update_y"].fillna(merged_df["update_x"])

# Drop the columns
merged_df = merged_df.drop(columns=["resistered_x", "resistered_y", "modified_x", "modified_y", "update_x", "update_y"])
merged_df

,career_info_x,company_name_x,company_sn_x,deadline_x,education_info_x,is_headhunting_x,job_idx,job_link_x,job_sectors_x,job_title_x,...,education_info_y,is_headhunting_y,job_link_y,job_sectors_y,job_title_y,support_date_y,work_place_y,resistered,modified,update
0,5 ~ 10년 · 정규직,한스컨설팅,Zkc5dWtSYXhYSURDVTd0c0tEMStVQT09,3일 전 등록,대학교(4년)↑,True,47846782,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"['SAP', 'ABAP', 'ERP', '솔루션']","강북 위치 제약중견기업 전산팀 대리-과장급, SAP 필수",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-24 19:58:11,NaT,2024-03-27 19:58:11.000000
1,3 ~ 10년 · 정규직,한스컨설팅,Zkc5dWtSYXhYSURDVTd0c0tEMStVQT09,3일 전 등록,대학교(4년)↑,True,47846790,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"['보안컨설팅', '정보보안', '모의해킹', '방화벽', 'ISMS']","코스피 상장, 판교 위치 화장품전문기업 IT 정보보안 대리-과장",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-24 19:58:11,NaT,2024-03-27 19:58:11.000000
2,3 ~ 10년 · 정규직,한스컨설팅,Zkc5dWtSYXhYSURDVTd0c0tEMStVQT09,3일 전 등록,대학교(4년)↑,True,47846794,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"['PL(프로젝트리더)', 'PM(프로젝트매니저)', 'PMO', 'PO(프로덕트오...","코스피 상장, 판교 위치 화장품전문기업 IT 기획 대리-과장급",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-24 19:58:11,NaT,2024-03-27 19:58:11.000000
3,3 ~ 8년 · 정규직,한스컨설팅,Zkc5dWtSYXhYSURDVTd0c0tEMStVQT09,3일 전 등록,대학교(4년)↑,True,47846799,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"['SAP', 'PO(프로덕트오너)', 'SCM']","코스피 상장, 판교 위치 화장품전문기업 SAP SCM 관리",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-24 19:58:11,NaT,2024-03-27 19:58:11.000000
4,3 ~ 10년 · 정규직,한스컨설팅,Zkc5dWtSYXhYSURDVTd0c0tEMStVQT09,3일 전 등록,대학교(4년)↑,True,47846809,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"['기술지원', 'CISO', 'DBA', '네트워크', '정보통신']","코스닥 상장, 클라우드기업 네트웍엔지니어 대리-과장, 이천근무",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-24 19:58:11,NaT,2024-03-27 19:58:11.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007,NaN,NaN,NaN,NaN,NaN,NaN,47875797,NaN,NaN,NaN,...,학력무관,True,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[데이터엔지니어, 솔루션, 헬스케어, Java, PL/SQL]",[국내유명 전략 컨설팅사] DB 엔지니어 채용,D-3,서울 서초구,2024-03-27 20:02:45.800208,NaT,2024-03-27 20:56:45.800208
2008,NaN,NaN,NaN,NaN,NaN,NaN,47875812,NaN,NaN,NaN,...,학력무관,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[백엔드/서버개발, 정보보안, 네트워크, 인프라, 클라우드]",DevOps 개발자 (2년 이상),채용시,서울 관악구,2024-03-27 20:13:45.800208,NaT,2024-03-27 20:56:45.800208
2009,NaN,NaN,NaN,NaN,NaN,NaN,47875822,NaN,NaN,NaN,...,학력무관,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[웹개발, 정보보안, SE(시스템엔지니어), 네트워크, 블록체인]",시스템 소프트웨어 개발자 (2년 이상),채용시,서울 관악구,2024-03-27 20:21:45.800208,NaT,2024-03-27 20:56:45.800208
2010,NaN,NaN,NaN,NaN,NaN,NaN,47875849,NaN,NaN,NaN,...,"대학(2,3년)↑",True,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[웹개발, Java, JSP, MySQL, 유지보수]","웹 개발자 모집 ( 문구류 제조 그룹사 / 대리,과장급 )",D-4,서울 서초구,2024-03-27 20:32:45.800208,NaT,2024-03-27 20:56:45.800208


# 새 시트로 덮어쓰기

In [ ]:
set_with_dataframe(sh, df)